## Qt alkalmazások készítése
![Alkalmazás](glider.png)

## Qt install

Ehhez a leckéhez már fontos, hogy mindenkinek python3 legyen a gépén

https://www.qt.io/download

## Minimal demo

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication


def main():
    app = QApplication(sys.argv)
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()


## Ablak

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget

# ...

w = QWidget()
w.show()


Beállítjuk az ablak nevét

In [ ]:
w.setWindowTitle("Game of Life")

Beállítjuk az ablak méretét

In [ ]:
w.resize(400, 400) 

Beállítjuk az ablak pozícióját

In [ ]:
w.move(0, 0)

További infók a QWidgetről:
http://doc.qt.io/qt-5/qwidget.html

## Saját QWidget
Ahhoz fog kelleni, hogy rajzolhassunk az ablakra

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget

class GameOfLifeWidget(QWidget):
    pass

def main():
    app = QApplication(sys.argv)
    w = GameOfLifeWidget()
    sys.exit(app.exec_())


## Rajzolás

In [ ]:
from PyQt5.QtGui import QPainter

class GameOfLifeWidget(QWidget):
    def paintEvent(self, e):
        qp = QPainter()
        qp.begin(self)
        # draw something
        qp.end()


FELADAT: mindig kíirni egy counter értékét, hogy lássuk, mikor van meghíva

## Vonalak rajzolása

In [ ]:
from PyQt5.QtGui import QPainter, QPen
from PyQt5.QtCore import Qt, QPoint

class GameOfLifeWidget(QWidget):
    def paintEvent(self, e):
        qp = QPainter()
        qp.begin(self)
        pen = QPen(Qt.gray, 2)
        qp.setPen(pen)
        for x in range(21):
            qp.drawLine(QPoint(x * 20, 0), QPoint(x * 20, 400))
        for y in range(21):
            qp.drawLine(QPoint(0, y * 20), QPoint(400, y * 20))
        qp.end()

# ...

## Téglalap

In [ ]:
from PyQt5.QtGui import QPainter, QBrush, QColor
from PyQt5.QtCore import Qt, QRect

class GameOfLifeWidget(QWidget):
    def paintEvent(self, e):

        qp = QPainter()
        qp.begin(self)
        brush = QBrush(Qt.SolidPattern)
        brush.setColor(QColor(0, 0, 0))
        qp.setBrush(brush)
        qp.drawRect(QRect(0, 0, 400, 400))
        qp.end()

# ...

## Sakktábla kirajzolása

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QBrush, QColor
from PyQt5.QtCore import Qt, QRect

class GameOfLifeWidget(QWidget):

    def paintEvent(self, e):
        qp = QPainter()
        qp.begin(self)

        brush = QBrush(Qt.SolidPattern)
        brush.setColor(QColor(0, 0, 0))
        qp.setBrush(brush)
        qp.drawRect(QRect(0, 0, 400, 400))

        brush.setColor(QColor(255,255,255))
        qp.setBrush(brush)
        for x in range(20):
            for y in range(20):
                if (x + y) % 2 == 0:
                    qp.drawRect(QRect(x * 20 + 1, y * 20 + 1, 18, 18))

        qp.end()


További infók a rajzolással kapcsolatban: http://doc.qt.io/qt-5/qpainter.html

## Sejtek beolvasása szöveges fájlból

In [ ]:
def cell_read(file_name):
    lines = []
    try:
        with open(file_name, "r") as file:
            lines = file.readlines()
    except FileNotFoundError:
        pass

    cells = []
    y = 0
    for line in lines:
        x = 0
        for cell in line.split():
            if cell == '1':
                cells.append((x, y))
            x += 1
        y += 1
    return cells

## Sejtek átadása a widgetünknek

Át kell írnunk a widgetünk konstruktorát. Ha az eredeti működést szeretnénk lemásolni, valami ilyesmit kapunk:

In [ ]:
class GameOfLifeWidget(QWidget):
    def __init__(self):
        super(QWidget, self).__init__()

Ezt fogjuk egy kicsit kibővíteni:

In [ ]:
class GameOfLifeWidget(QWidget):
    def __init__(self, cells):
        super(QWidget, self).__init__()
        self.cells = cells

és a megfelelő helyen, az inicializáláskor átadjuk a sejteket

In [ ]:
def main():
    cells = cell_read("glider.txt")
    w = GameOfLifeWidget(cells)
    # ...

...és kirajzoljuk őket:

In [ ]:
class GameOfLifeWidget(QWidget):
    def paintEvent(self, e):
        qp = QPainter()
        qp.begin(self)

        brush = QBrush(Qt.SolidPattern)
        brush.setColor(QColor(0, 0, 0))
        qp.setBrush(brush)
        qp.drawRect(QRect(0, 0, 400, 400))

        for x in range(20):
            for y in range(20):
                g = (255 if (x, y) in self.cells else 0)
                brush.setColor(QColor(g, g, g))
                qp.setBrush(brush)
                qp.drawRect(QRect(x * 20 + 1, y * 20 + 1, 18, 18))

        pen = QPen(Qt.gray, 2)
        qp.setPen(pen)
        for x in range(21):
            qp.drawLine(QPoint(x * 20, 0), QPoint(x * 20, 400))
        for y in range(21):
            qp.drawLine(QPoint(0, y * 20), QPoint(400, y * 20))
        qp.end()


HÁZI FELADAT: Utánanézni, hogyan lehet png-ket kirajzolni, vagy valahogy szebbé tenni a kinézetét a sejteknek.

## Gomb az interakciókhoz
Az alábbi módszerrel adunk hozzá az ablakunkhoz egy gombot

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton

class GameOfLifeWidget(QWidget):
    def __init__(self, cells):

        # ...

        self.stepbutton = QPushButton('Step', self)
        self.stepbutton.clicked.connect(self.takeOneStep)
        self.stepbutton.resize(100, 50)
        self.stepbutton.move(300, 0)

Ezután meg kell mondanunk, hogy a takeOneStep függvényben mit csináljon

In [ ]:
from conway import GameOfLife

class GameOfLifeWidget(QWidget):
    
    # ...

    def takeOneStep(self):
        gol = GameOfLife()
        self.cells = gol.evolve(self.cells)
        self.update()

HÁZI FELADAT: Back gomb - visszalép egyet (meg kell jegyezni a cellák fejlődésének történetét), de csak az alapállapotig. Lehetséges visszalépni az alapállapot előtti állapotba is?

## Timer
Avagy hogyan fejlődjenek a sejtjeink maguktól. Ehhez szükségünk lesz egy timerre:

In [ ]:
from PyQt5.QtCore import QRect, Qt, QPoint, QTimer

# ...
class GameOfLifeWidget(QWidget):

    def __init__(self, cells):

        # ...

        self.paused = True
        self.timer = QTimer()
        self.timer.timeout.connect(self.stepTimer)
        self.timer.start(100)

és meg kell mondanunk, hogy mit kell ismételnünk:

In [ ]:
class GameOfLifeWidget(QWidget):

    # ...
    
    def stepTimer(self):
        if not self.paused:
            self.takeOneStep()


## Play/Pause gomb
hogy bármikor el tudjuk indítani és megállítani az animációnkat.

In [ ]:
class GameOfLifeWidget(QWidget):
    def __init__(self, cells):

        # ...

        self.playbutton = QPushButton('Play', self)
        self.playbutton.clicked.connect(self.clickedPlay)
        self.playbutton.resize(100, 50)
        self.playbutton.move(200, 0)


Ha még nem történik evolúció, akkor el kell indítanunk.

Ha már történik evolúció, akkor meg kell állítanunk.

In [ ]:
class GameOfLifeWidget(QWidget):

    # ...
    
    def clickedPlay(self):
        if self.paused:
            self.paused = False
            self.playbutton.setText("Pause")
        else:
            self.paused = True
            self.playbutton.setText("Play")


HÁZI FELADAT: Kigenerálni egy hosszabb lépéssorozatot, és akár visszafelé is le lehessen játszani egy külön gombbal, csúszkával akármelyik pontjára vissza tudjunk lépni az evolúciónak

## Egérkattintás
Ahhoz, hogy hozzá tudjunk adni sejteket, és el tudjunk venni, muszáj valahogy érzékelnünk, hol történt kattintás:

In [ ]:
class GameOfLifeWidget(QWidget):

    # ...
    
    def mousePressEvent(self, event):
        posx = int(event.x() / 20)
        posy = int(event.y() / 20)
        if (posx, posy) in self.cells:
            index = self.cells.index((posx, posy))
            del self.cells[index]
        else:
            self.cells.append((posx, posy))
        self.update()